In [2]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)
# 저장된 rag_data.pickle을 읽기 모드로 열기

In [3]:
import pandas as pd

df = pd.DataFrame(rag_data)
df

,questions,contexts,contexts_answer_idx,contexts_answers,answers
0,글로벌 저금리 현상이 부각된 원인은 무엇인가요?,[... 변화의 동인은 크게 두 가지로 분류할\n수 있다. 하나는 금융업계 전반에 ...,2,2. 뉴노멀의 도래(신 3저의 도래)\n최근 코로나 사태를 포함하여 2008년 글로...,"글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융..."
1,고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.,"[... 둔갑하여 발생하였다. 쏠림현상을 해결할 수 있는 방법은, 원론적인 이\n야...",1,축소되어 자산운용의 어려움이 가중\n∙ 투자자 입장에서는 원하는 수익률을 얻기 어려...,고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...
2,20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...,[1. 디지털화의 진전과 금융투자업의 변화 : 20세기 이후\n가. 1950~60년...,0,1. 디지털화의 진전과 금융투자업의 변화 : 20세기 이후\n가. 1950~60년대...,20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...
3,2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?,"[∙ 또한, 투자은행의 명성에 의존하는 대표적인 업무인 IPO, M&A\n등도 전산...",2,"∙ 또한, 투자은행의 명성에 의존하는 대표적인 업무인 IPO, M&A\n등도 전산화...","2000년대 이후 글로벌 IB들은 스스로 전산화를 추진하였으며, 이와 밀접하게 연관..."
4,금융시스템 개혁법은 언제 제정되었나요?,[... 금융소비자보호법 제정안 및 금융위설치법 개정안 141에 대해서 2. 주요...,2,및 일본 내 해외투자 수요를 포용하는데 적극적\n일련의 변화과정에서 일본 금융당국의...,금융시스템 개혁법은 1998년에 제정되었습니다.
...,...,...,...,...,...
195,우리나라 은행들이 기업대출을 통해 외부자금을 필요로 하는 산업에 어떻게 기여하였나요?,"[... 주무관청대신은\n업종에 따라 경제산업성(제조업, 유통·소매업), 금융청(금...",2,... 긍정적인 측면과 부정적인 측면\n∙ 우리나라 은행의 기업대출이 사업체 수 증...,우리나라 은행들은 기업대출을 통해 외부자금을 필요로 하는 산업에 자금을 효율적으로 ...
196,은행의 기업대출이 중소기업의 부가가치 성장에 어떻게 기여하였는지 설명해주세요.,[기존 중개업자가 수행하는 업무는 정의에서 제외되어 있기 때문에 기존\n중개업자가 ...,1,... 데\n은행의 기업대출 대부분도 중소기업대출이어서 과연 은행이 중소기업대출을\...,은행의 기업대출이 중소기업의 부가가치 성장에 기여하는 방식은 크게 두 가지로 볼 수...
197,기업 저축의 역설에 대해 설명해주세요.,[... 개업 등록을 받을 수 없다. 따라서 예를 들어 은행대리업자가 예금 등 매개...,2,... 현재의 기업 저축률은 불필요하게 높은 수준일 가능성을 시사\n4. 2004년...,"기업 저축의 역설이란, 기업들이 지나치게 많은 자금을 저축하면서 경제의 역동성이 상..."
198,명목 경제 성장률이 높은 것이 경기호황을 의미하는 이유는 무엇인가요?,[... 비해 명목 성장률이 특별히 높았던 기간이 아니었다. 50) 먼저 2001...,0,... 비해 명목 성장률이 특별히 높았던 기간이 아니었다. 50) 먼저 2001년...,명목 경제 성장률이 높다는 것은 경기가 호황인 상태를 의미합니다. 이는 경제가 활발...


In [ ]:
rag_data['questions'][0]
#rag_data의 'questions'열의 첫번째 행의 값

'글로벌 저금리 현상이 부각된 원인은 무엇인가요?'

In [12]:
rag_data['contexts'][0][1]

'3. 문제점\n1) 기업신용위험 평가항목의 적정성\n글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로\n금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을\n측정하는 데 적정한 지표인지에 대한 검토가 필요\n기업신용위험 정 ... 작용할 것으로 예상\n저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배\n율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이\n발생\n더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기\n연장 및 이자상환 유예조치로 이 ... 는 추세\n이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이\n있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...'

In [13]:
rag_data['contexts'][0][0]

'... 변화의 동인은 크게 두 가지로 분류할\n수 있다. 하나는 금융업계 전반에 부각되는 디지털화(digitalization)1)이고\n다른 하나는 저금리·저성장으로 대표되는 뉴노멀(new normal)이다.  ... . 뉴노멀의 특징으로는 금융투자\n및 자산운용의 어려움이 가중되는 경향, 고수익-고위험 부문으로의 쏠림\n현상 등의 위험요인과 함께 자산관리서비스 수요의 증가 등 대응 노력을\n살펴본다. 제III장은 디지털화 및 뉴노멀에 대응하는 해외 사례들을 살펴본다.  ... 뉴노멀의\n주요 특징인 저성장, 저금리 등은 일본의 금융투자업계가 30년 전부터 고민해\n온 문제이기 때문이다.  ... 제III장에서 살펴본 해외\n사례들에서 살펴본 금융투자업계 변화의 규칙성이 어느 정도 유지될 것이라는\n1 디지털화(digitalization)에 대한 ...'

In [14]:
from utils import get_embedding, cosine_similarity

embed_q = get_embedding(rag_data['questions'][0])
embed_c0 = get_embedding(rag_data['contexts'][0][0])
embed_c1 = get_embedding(rag_data['contexts'][0][1])
embed_c2 = get_embedding(rag_data['contexts'][0][2])
#Dataframe의 각 데이터를 임베딩으로 변환

In [15]:
print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))
#question 임베딩과 context 임베딩 사이의 코사인 유사도를 구함.

0.36210903416669354
0.44104521085802917
0.4154599195871476


In [16]:
rag_data['contexts_answer_idx'][0]
#'context_answer_idx' column 의 첫번째 행의 값을 64bit 정수 형태로 반환

np.int64(2)

In [17]:
embed_q = get_embedding(rag_data['questions'][0], model='text-embedding-3-large')
embed_c0 = get_embedding(rag_data['contexts'][0][0], model='text-embedding-3-large')
embed_c1 = get_embedding(rag_data['contexts'][0][1], model='text-embedding-3-large')
embed_c2 = get_embedding(rag_data['contexts'][0][2], model='text-embedding-3-large')
#Dataframe의 각 데이터를 임베딩으로 변환하는데, 모델을 지정하여 수행

In [18]:
print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))
#question 임베딩과 context 임베딩 사이의 코사인 유사도를 구함.

0.4088359818172236
0.39288325841152233
0.5339717607089124


In [ ]:
from tqdm import tqdm

num_questions = 10
num_contexts = 3

top_context_indices = []
#"top_context_indices"라는 empty list 만들어줌.

for i in tqdm(range(num_questions)):
  embed_q = get_embedding(rag_data['questions'][i])
  # i = 0 에서 9까지 반복하면서 "embed_q"에 ['questions'][i]를 임베딩으로 변환한 값을 저장.

  similarities = []
  #"similarities"라는 empty list 만들어줌.
  for j in range(num_contexts):
    embed_c = get_embedding(rag_data['contexts'][i][j])
    #['contexts'][i]에 있는 각 데이터의 임베딩을 순차적으로 embed_c에 저장(j = 0 에서 2까지)
    similarities.append(cosine_similarity(embed_q, embed_c))
    #embed_q와 embed_c 사이의 코사인 유사도를 구하여 similarities list에 append. for문이 끝나면 
    # similarities는 item 세 개짜리 리스트가 됨.

  top_context_index = similarities.index(max(similarities))
  # similarities list에서 최댓값을 찾은 후, 최댓값에 해당되는 인덱스를 반환.
  top_context_indices.append(top_context_index)
  # 각 행의 context 중 question과의 관련도가 가장 높은 데이터의 인덱스가 저장됨.

print(f"Top context indices: {top_context_indices}")

100%|██████████| 10/10 [00:25<00:00,  2.59s/it]

Top context indices: [1, 1, 0, 1, 1, 1, 2, 0, 1, 1]


In [ ]:
rag_data['contexts_answer_idx'][:num_questions]
#정답이 되는 인덱스 반환

[np.int64(2),
 np.int64(1),
 np.int64(0),
 np.int64(2),
 np.int64(2),
 np.int64(1),
 np.int64(2),
 np.int64(2),
 np.int64(1),
 np.int64(1)]

In [ ]:
def calculate_accuracy(predicted, actual):
  correct = sum (p == a for p, a in zip(predicted, actual))
  #zip(p,a): p와 a의 요소를 하나씩 묶어 튜플 형성.
  # if p == a then correct ++
  total = len(predicted)

  accuracy = correct / total
  # p == a 인 items / 전체 items
  return accuracy

accuracy = calculate_accuracy(top_context_indices, rag_data['contexts_answer_idx'][:10])
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 60.00%


### 정답 Context가 없는 경우
- RAGAS
  - GPT-4를 활용한 RAG 로직 평가용 라이브러리
    - RAG 뿐만 아니라 LLM 단독 그리고 LLM + RAG 평가도 가능
  - RAG 평가의 경우 정답 Context가 있어도 사용이 가능하지만 정답이 없는 경우에도 사용이 가능
  - OpenAI에서도 RAG 로직 평가를 위해 사용 (OpenAI Dev Day)
  - 기본으로 제공하는 평가용 System Prompt가 영문 기반이라 한글 평가는 상대적으로 정확도가 약간 떨어지는 편

In [23]:
contexts_predictions = []
for i in range(len(top_context_indices)):
    index = top_context_indices[i]
    contexts_predictions.append([rag_data['contexts'][i][index]])
contexts_predictions
# contexts_predictions list: 위에서 만든 top_context_indices의 인덱스에 해당하는 각 context 반환 후 저장된 list

[['3. 문제점\n1) 기업신용위험 평가항목의 적정성\n글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로\n금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을\n측정하는 데 적정한 지표인지에 대한 검토가 필요\n기업신용위험 정 ... 작용할 것으로 예상\n저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배\n율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이\n발생\n더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기\n연장 및 이자상환 유예조치로 이 ... 는 추세\n이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이\n있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...'],
 ['축소되어 자산운용의 어려움이 가중\n∙ 투자자 입장에서는 원하는 수익률을 얻기 어려워 보다 높은 수익률을\n추구하는 투자행태 ... 나. 고수익-고위험 부문으로 쏠림현상\n금융회사, 가계 등의 수익률 추구 성향(search for yield)이 강화되어\n위험자산으로 과도한 자금 유입이 발생할 수 있음.\n∙ 저금리 기조 하에서는 투자자의 고수익 추구로 인해 고위험 파생상\n품, 부동산금융 등 리스크가 큰 분야로의 쏠림현상 심화 가능성\n∙ 사모펀드 부문의 빠른 성장, 파생상품 자산의 급증 등 개인의 투자위\n험도 증가하는 추세\n* 고수익-고위험 추구  ...  등)으로 기대수익률을 충족시키지\n못해 고위험-고수익을 추구할 가능성 증가\n∙ 개인 투자자들은 이에 대한 전문성 부족으로  ...'],
 ['1. 디지털화의 진전과 금융투자업의 변화 : 20세기 이후\n가. 1950~60년대\nMorrison and Wilhelm(2007), Jensen(1993) 등은 컴퓨터 기술의\n발전과 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 큰\n영향을 끼치는 것은 20세기 중반 이후로 평가\n1960년대 후반 소위 백오피스 위기를 거치면서 백오피스 업무, 거래소\n거래 중심의 전산화가 가속 ... 1980년대 이후 

In [ ]:
print(contexts_predictions)
print(rag_data['questions'][:10])
print(rag_data['answers'][:10])
print(rag_data['contexts'][:10])
print(rag_data['contexts_answers'][:10])

In [21]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
# "gpt-4o" 모델 이용하여 evaluator_llm 지정.


In [ ]:
from ragas.metrics import LLMContextRecall
from ragas import evaluate
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset

metrics = [
    LLMContextRecall(llm=evaluator_llm), 
]
# LLMContextRecall: 컨텍스트 품질 평가하는 matric.
# evaluateor_llm: 평가에 사용되는 LLM

samples = []
for question, context, answer in zip(rag_data['questions'][:num_questions], contexts_predictions, rag_data['answers'][:num_questions]):
    # zip()을 통해 rag_data['questions'][:num_questions], contexts_predictions, rag_data['answers'][:num_questions]에서 한 아이템 씩 가져와
    #(question, context, answer) 튜플로 저장.
    sample = SingleTurnSample(
        user_input=question,
        reference=answer,
        retrieved_contexts=context 
    )
    # SingleTurnSample: 평가 위한 샘플 데이터 구조 정의하는 class. question, answer, context로 구성됨.
    samples.append(sample)
    # 빈 리스트 samples에 각 sample append.
dataset = EvaluationDataset(samples)
# samples list를 EvaluationDataset 객체로 변환. 
score = evaluate(dataset, metrics=metrics)
# dataset를 metrics로 평가하여 evaluation score반환.
score


Evaluating: 100%|██████████| 10/10 [00:15<00:00,  1.53s/it]


{'context_recall': 0.6850}